[View in Colaboratory](https://colab.research.google.com/github/SaffetGokcenSen/seq2seq_keras_test_colab/blob/master/seq2seq_keras_colab.ipynb)

In [7]:
!pip install numpy==1.14.5
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

    100% |████████████████████████████████| 12.2MB 4.0MB/s 
  Found existing installation: numpy 1.15.1
    Uninstalling numpy-1.15.1:
      Successfully uninstalled numpy-1.15.1


In [1]:
from google.colab import files
uploaded = files.upload()

Saving tur.txt to tur.txt


In [0]:
batch_size = 64  # Batch size for training.
epochs = 22  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'tur.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [3]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 84
Max sequence length for inputs: 15
Max sequence length for outputs: 43


In [0]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [0]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [0]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [11]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,
          verbose=2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/22
 - 19s - loss: 1.0775 - val_loss: 1.0924
Epoch 2/22
 - 17s - loss: 0.8508 - val_loss: 0.8876
Epoch 3/22
 - 17s - loss: 0.7199 - val_loss: 0.7802
Epoch 4/22
 - 16s - loss: 0.6538 - val_loss: 0.7309
Epoch 5/22
 - 17s - loss: 0.6110 - val_loss: 0.6995
Epoch 6/22
 - 16s - loss: 0.5779 - val_loss: 0.6672
Epoch 7/22
 - 17s - loss: 0.5506 - val_loss: 0.6355
Epoch 8/22
 - 17s - loss: 0.5265 - val_loss: 0.6157
Epoch 9/22
 - 16s - loss: 0.5048 - val_loss: 0.5987
Epoch 10/22
 - 17s - loss: 0.4856 - val_loss: 0.5818
Epoch 11/22
 - 16s - loss: 0.4674 - val_loss: 0.5676
Epoch 12/22
 - 16s - loss: 0.4513 - val_loss: 0.5576
Epoch 13/22
 - 17s - loss: 0.4355 - val_loss: 0.5506
Epoch 14/22
 - 16s - loss: 0.4211 - val_loss: 0.5406
Epoch 15/22
 - 16s - loss: 0.4076 - val_loss: 0.5353
Epoch 16/22
 - 16s - loss: 0.3941 - val_loss: 0.5286
Epoch 17/22
 - 17s - loss: 0.3815 - val_loss: 0.5214
Epoch 18/22
 - 17s - loss: 0.3695 - val_loss: 0.5193
Epoch 1